# Extracting entities from a Terms of Service document

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/extracting_entities.ipynb).

In this example, we will use Guardrails to extract key information from a Terms-of-Service document.

## Objective

We want to extract structured information about all fees and interest rates associated with the Chase credit card.

## Step 0: Download PDF and load it as string

To get started, download the document from [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/data/chase_card_agreement.pdf) and save it in `data/chase_card_agreement.pdf`.

Guardrails has some built-in functions to help with common tasks. Here, we will use the `read_pdf` function to load the PDF as a string.

In [1]:
import guardrails as gd

from rich import print

content = gd.docs_utils.read_pdf("data/chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

Chase Credit Card Document:

2/25/23, 7:59 PM about:blank
about:blank 1/4
PRICING INFORMATION
INTEREST RATES AND INTEREST CHARGES
Purchase Annual
Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on th
...

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

Here, we request:

1. A list of the fees associated with the card. We ask for sub-information, each with its own quality criteria and corrective action.
2. A object (i.e. key-value pairs) for the interest.

In [3]:
rail_str = """
<rail version="0.1">

<output>

    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="lower-case; two-words" on-fail-lower-case="noop" on-fail-two-words="reask"/>
            <string name="explanation" format="one-line" on-fail-one-line="noop" />
            <float name="value" format="percentage"/>
        </object>
    </list>
    <object name="interest_rates" description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?" />
</output>


<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

{{document}}

@xml_prefix_prompt

{output_schema}

@json_suffix_prompt_v2_wo_none</prompt>

</rail>
"""

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [4]:
guard = gd.Guard.from_rail_string(rail_str)

/Users/shreyarajpal/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter 1-indexed is not valid for element integer.
  warnings.warn(
/Users/shreyarajpal/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter percentage is not valid for element float.
  warnings.warn(


As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

We see the prompt that will be sent to the LLM. The `{document}` is substituted with the user provided value at runtime.

In [5]:
print(guard.base_prompt)

Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
'None'.

{document}


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>

    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed"/>
            <string name="name" format="lower-case; two-words"/>
            <string name="explanation" format="one-line"/>
            <float name="value" format="percentage"/>
        </object>
    </list>
    <object name="interest_rates" description="What are the interest rates offered by the bank on savings and 
checking accounts, loans, and credit products?"/>
</output>





ONLY return a valid JSON object (no other text is necessary). The JSON MUST conform to the XML format, including 
any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise.

JSON Output:

## Step 3: Wrap the LLM API call with `Guard`

In [6]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={"document": content[:6000]},
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [9]:
print(validated_response)

{
    'fees': [
        {'index': 1, 'name': 'annual membership', 'explanation': 'none', 'value': 0},
        {
            'index': 2,
            'name': 'fee one',
            'explanation': 'monthly fee of 0% of the amount of each eligible purchase transaction or amount 
selected to create a My Chase Plan while in the 0% Intro Purchase APR period. After that, monthly fee of 1.72% of 
the amount of each eligible purchase transaction or amount selected to create a My Chase Plan.',
            'value': 1.72
        },
        {
            'index': 3,
            'name': 'balance transfers',
            'explanation': 'intro fee of either $5 or 3% of the amount of each transfer, whichever is greater, on 
transfers made within 60 days of account opening. After that: Either $5 or 5% of the amount of each transfer, 
whichever is greater.',
            'value': 5
        },
        {
            'index': 4,
            'name': 'cash advances',
            'explanation': 'either $10 or 5% of the amount of each transaction, whichever is greater.',
            'value': 5
        },
        {
            'index': 5,
            'name': 'foreign transactions',
            'explanation': '3% of the amount of each transaction in U.S. dollars.',
            'value': 3
        },
        {'index': 6, 'name': 'late payment', 'explanation': 'up to $40.', 'value': 0},
        {'index': 7, 'name': 'fee two', 'explanation': 'none', 'value': 0},
        {'index': 8, 'name': 'return payment', 'explanation': 'up to $40.', 'value': 0},
        {'index': 9, 'name': 'return check', 'explanation': 'none', 'value': 0}
    ],
    'interest_rates': {
        'purchase': {
            'apr': 0,
            'explanation': '0% Intro APR for the first 18 months that your Account is open. After that, 19.49%. 
This APR will vary with the market based on the Prime Rate.'
        },
        'my_chase_loan': {
            'apr': 19.49,
            'explanation': '19.49%. This APR will vary with the market based on the Prime Rate.'
        },
        'balance_transfer': {
            'apr': 0,
            'explanation': '0% Intro APR for the first 18 months that your Account is open. After that, 19.49%. 
This APR will vary with the market based on the Prime Rate.'
        },
        'cash_advance': {
            'apr': 29.49,
            'explanation': '29.49%. This APR will vary with the market based on the Prime Rate.'
        },
        'penalty': {
            'apr': 29.99,
            'explanation': 'Up to 29.99%. This APR will vary with the market based on the Prime Rate.'
        },
        'prime_rate': 7.75
    }
}